# Visualización: proyecto

## Conjuntos de datos

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from plotnine import *
import re

### `df_p_activa`

In [2]:
df_p_activa = pd.read_csv('../data/empleo/datasets-ISTAC_eparegActiva-1921.csv')

# Eliminamos las columnas sobrantes
df_p_activa.drop(
    columns = [
        'ESTADO_OBSERVACION#es',
        'CONFIDENCIALIDAD_OBSERVACION#es',
        'NOTAS_OBSERVACION#es'
    ],
    inplace = True
)

# Eliminamos las entradas con SEXO_CODE == _T (Total)
df_p_activa.drop(
    df_p_activa[df_p_activa['SEXO_CODE'] == '_T'].index,
    inplace = True
)

# Eliminamos las entradas de cada isla
rows_islas = df_p_activa[df_p_activa['TERRITORIO_CODE'].str.contains(r'ES\d{3}')].index
df_p_activa.drop(
    rows_islas,
    inplace = True
)

# Renombramos la columna 'OBS_VALUE' a 'FREQUENCY'
df_p_activa.rename(
    columns = {'OBS_VALUE': 'FREQUENCY'},
    inplace = True
)

display(df_p_activa)

,TIME_PERIOD#es,TIME_PERIOD_CODE,TERRITORIO#es,TERRITORIO_CODE,SEXO#es,SEXO_CODE,EDAD#es,EDAD_CODE,SITUACION_LABORAL_REGISTRADA#es,SITUACION_LABORAL_REGISTRADA_CODE,FREQUENCY
0,2021 Cuarto trimestre,2021-Q4,Artenara,35005,Mujeres,F,De 35 a 44 años,Y35T44,Personas desempleadas registradas,UNE_REG,13
1,2021 Cuarto trimestre,2021-Q4,Artenara,35005,Mujeres,F,16 años o más,Y_GE16,Personas desempleadas registradas,UNE_REG,47
2,2021 Cuarto trimestre,2021-Q4,Artenara,35005,Mujeres,F,De 16 a 24 años,Y16T24,Personas desempleadas registradas,UNE_REG,5
3,2021 Cuarto trimestre,2021-Q4,Artenara,35005,Mujeres,F,55 años o más,Y_GE55,Personas desempleadas registradas,UNE_REG,15
4,2021 Cuarto trimestre,2021-Q4,Artenara,35005,Mujeres,F,De 25 a 34 años,Y25T34,Personas desempleadas registradas,UNE_REG,5
...,...,...,...,...,...,...,...,...,...,...,...
62197,2019 Primer trimestre,2019-Q1,Agulo,38002,Hombres,M,16 años o más,Y_GE16,Personas activas registradas,ACT_REG,262
62198,2019 Primer trimestre,2019-Q1,Agulo,38002,Hombres,M,De 16 a 24 años,Y16T24,Personas activas registradas,ACT_REG,13
62199,2019 Primer trimestre,2019-Q1,Agulo,38002,Hombres,M,55 años o más,Y_GE55,Personas activas registradas,ACT_REG,66
62200,2019 Primer trimestre,2019-Q1,Agulo,38002,Hombres,M,De 25 a 34 años,Y25T34,Personas activas registradas,ACT_REG,48


### `df_p_ocup_actividad`

In [3]:
df_p_ocup = pd.read_csv('../data/empleo/datasets-ISTAC_eparegOcupadaActividad-1921.csv')

# Eliminamos las columnas sobrantes
df_p_ocup.drop(
    columns = [
        'ESTADO_OBSERVACION#es',
        'CONFIDENCIALIDAD_OBSERVACION#es',
        'NOTAS_OBSERVACION#es'
    ],
    inplace = True
)

# Eliminamos las entradas con SEXO_CODE == _T (Total)
df_p_ocup.drop(df_p_ocup[df_p_ocup['SEXO_CODE'] == '_T'].index, inplace = True)

# Eliminamos las entradas de cada isla
rows_islas = df_p_ocup[df_p_ocup['TERRITORIO_CODE'].str.contains(r'ES\d{3}')].index
df_p_ocup.drop(
    rows_islas,
    inplace = True
)

# Renombramos la columna 'OBS_VALUE' a 'FREQUENCY'
df_p_ocup.rename(
    columns = {'OBS_VALUE': 'FREQUENCY'},
    inplace = True
)

display(df_p_ocup)

,TIME_PERIOD#es,TIME_PERIOD_CODE,SEXO#es,SEXO_CODE,TERRITORIO#es,TERRITORIO_CODE,ACTIVIDAD_ECONOMICA#es,ACTIVIDAD_ECONOMICA_CODE,FREQUENCY
0,2021 Cuarto trimestre,2021-Q4,Mujeres,F,Artenara,35005,Servicios,A4_04,156
1,2021 Cuarto trimestre,2021-Q4,Hombres,M,Artenara,35005,Servicios,A4_04,201
3,2021 Tercer trimestre,2021-Q3,Mujeres,F,Artenara,35005,Servicios,A4_04,152
4,2021 Tercer trimestre,2021-Q3,Hombres,M,Artenara,35005,Servicios,A4_04,198
6,2021 Segundo trimestre,2021-Q2,Mujeres,F,Artenara,35005,Servicios,A4_04,146
...,...,...,...,...,...,...,...,...,...
20728,2019 Tercer trimestre,2019-Q3,Hombres,M,Agulo,38002,Total,_T,185
20730,2019 Segundo trimestre,2019-Q2,Mujeres,F,Agulo,38002,Total,_T,163
20731,2019 Segundo trimestre,2019-Q2,Hombres,M,Agulo,38002,Total,_T,217
20733,2019 Primer trimestre,2019-Q1,Mujeres,F,Agulo,38002,Total,_T,173


### `df_p_parados`

In [4]:
df_p_parados = pd.read_csv('../data/empleo/datasets-ISTAC_eparegParadosActividad-1921.csv')

# Eliminamos las columnas sobrantes
df_p_parados.drop(
    columns = [
        'ESTADO_OBSERVACION#es',
        'CONFIDENCIALIDAD_OBSERVACION#es',
        'NOTAS_OBSERVACION#es'
    ],
    inplace = True
)

# Eliminamos las entradas con SEXO_CODE == _T (Total)
df_p_parados.drop(df_p_parados[df_p_parados['SEXO_CODE'] == '_T'].index, inplace = True)

# Eliminamos las entradas de cada isla
rows_islas = df_p_parados[df_p_parados['TERRITORIO_CODE'].str.contains(r'ES\d{3}')].index
df_p_parados.drop(
    rows_islas,
    inplace = True
)

# Renombramos la columna 'OBS_VALUE' a 'FREQUENCY'
df_p_parados.rename(
    columns = {'OBS_VALUE': 'FREQUENCY'},
    inplace = True
)

display(df_p_parados)

,TIME_PERIOD#es,TIME_PERIOD_CODE,SEXO#es,SEXO_CODE,TERRITORIO#es,TERRITORIO_CODE,ACTIVIDAD_ECONOMICA#es,ACTIVIDAD_ECONOMICA_CODE,FREQUENCY
0,2021 Cuarto trimestre,2021-Q4,Hombres,M,Arrecife,35004,Servicios,A4_04,1691
1,2021 Cuarto trimestre,2021-Q4,Mujeres,F,Arrecife,35004,Servicios,A4_04,2837
2,2021 Tercer trimestre,2021-Q3,Hombres,M,Arrecife,35004,Servicios,A4_04,1983
3,2021 Tercer trimestre,2021-Q3,Mujeres,F,Arrecife,35004,Servicios,A4_04,3294
4,2021 Segundo trimestre,2021-Q2,Hombres,M,Arrecife,35004,Servicios,A4_04,3275
...,...,...,...,...,...,...,...,...,...
12667,2019 Tercer trimestre,2019-Q3,Mujeres,F,El Pinar de El Hierro,38901,Total,_T,79
12668,2019 Segundo trimestre,2019-Q2,Hombres,M,El Pinar de El Hierro,38901,Total,_T,74
12669,2019 Segundo trimestre,2019-Q2,Mujeres,F,El Pinar de El Hierro,38901,Total,_T,85
12670,2019 Primer trimestre,2019-Q1,Hombres,M,El Pinar de El Hierro,38901,Total,_T,76


### `df_p_tcontrato`

In [5]:
df_p_tcontrato = pd.read_csv('../data/empleo/datasets-ISTAC_eparegParadosActividad-1921.csv')

# Eliminamos las columnas sobrantes
df_p_tcontrato.drop(
    columns = [
        'ESTADO_OBSERVACION#es',
        'CONFIDENCIALIDAD_OBSERVACION#es',
        'NOTAS_OBSERVACION#es'
    ],
    inplace = True
)

# Eliminamos las entradas con SEXO_CODE == _T (Total)
df_p_tcontrato.drop(df_p_tcontrato[df_p_tcontrato['SEXO_CODE'] == '_T'].index, inplace = True)

# Eliminamos las entradas de cada isla
rows_islas = df_p_tcontrato[df_p_tcontrato['TERRITORIO_CODE'].str.contains(r'ES\d{3}')].index
df_p_tcontrato.drop(
    rows_islas,
    inplace = True
)

# Renombramos la columna 'OBS_VALUE' a 'FREQUENCY'
df_p_tcontrato.rename(
    columns = {'OBS_VALUE': 'FREQUENCY'},
    inplace = True
)

display(df_p_tcontrato)

,TIME_PERIOD#es,TIME_PERIOD_CODE,SEXO#es,SEXO_CODE,TERRITORIO#es,TERRITORIO_CODE,ACTIVIDAD_ECONOMICA#es,ACTIVIDAD_ECONOMICA_CODE,FREQUENCY
0,2021 Cuarto trimestre,2021-Q4,Hombres,M,Arrecife,35004,Servicios,A4_04,1691
1,2021 Cuarto trimestre,2021-Q4,Mujeres,F,Arrecife,35004,Servicios,A4_04,2837
2,2021 Tercer trimestre,2021-Q3,Hombres,M,Arrecife,35004,Servicios,A4_04,1983
3,2021 Tercer trimestre,2021-Q3,Mujeres,F,Arrecife,35004,Servicios,A4_04,3294
4,2021 Segundo trimestre,2021-Q2,Hombres,M,Arrecife,35004,Servicios,A4_04,3275
...,...,...,...,...,...,...,...,...,...
12667,2019 Tercer trimestre,2019-Q3,Mujeres,F,El Pinar de El Hierro,38901,Total,_T,79
12668,2019 Segundo trimestre,2019-Q2,Hombres,M,El Pinar de El Hierro,38901,Total,_T,74
12669,2019 Segundo trimestre,2019-Q2,Mujeres,F,El Pinar de El Hierro,38901,Total,_T,85
12670,2019 Primer trimestre,2019-Q1,Hombres,M,El Pinar de El Hierro,38901,Total,_T,76


### `df_defun`

In [6]:
df_defun = pd.read_csv('../data/defunciones/DefCanariaStatTasa.csv').drop(columns = 'Unnamed: 0')

# Rellenar los registros de puntos con 0
df_defun['Defunciones'].replace(
    to_replace = r'^\.+$',
    value = 0,
    regex = True,
    inplace = True
)

# Eliminar los posibles puntos de centena intermedios
df_defun['Defunciones'] = df_defun['Defunciones'].str.replace('.','')

# Sustituye las comas decimales por puntos decimales
df_defun['Defunciones'] = df_defun['Defunciones'].str.replace(',','.')

# Convertir el tipo de dato a numérico
df_defun['Defunciones'] = df_defun['Defunciones'].astype(float)

# Eliminar los espacios sobrantes en los municipios
df_defun['Municipio'] = df_defun['Municipio'].str.strip()

# Corregir los nombres mal ordenados: 'nombre, (pronombre)'
def fix_municipality_name(mun_name):
    match_result = re.match(
        pattern = r'(.*) \((.*)\)',
        string = mun_name
    )
    if match_result:
        name, pronoun = match_result.groups()
        return f'{pronoun} {name}'
    else:
        return mun_name
    
df_defun['Municipio'] = df_defun['Municipio'].apply(fix_municipality_name)

# Corregir los nombres mal escritos de los municipios
df_defun['Municipio'].replace(
    to_replace = 'La Laguna' ,
    value = 'San Cristóbal de La Laguna',
    inplace = True
)
df_defun['Municipio'].replace(
    to_replace  = df_defun['Municipio'].sort_values().unique(),
    value = df_p_tcontrato['TERRITORIO#es'].sort_values().unique(),
    inplace = True
)

# Añadir los códigos de municipio
cod_mun_dict = (df_p_activa[['TERRITORIO#es', 'TERRITORIO_CODE']]
                .drop_duplicates()
                .set_index('TERRITORIO#es')
                .to_dict()
                .get('TERRITORIO_CODE')
)
df_defun['Municipio_codigo'] = df_defun['Municipio'].apply(lambda x: cod_mun_dict[x])

display(df_defun)

,Municipio,Año,Sexo,Tipo,Defunciones,Municipio_codigo
0,Arrecife,2021,Hombres,I. CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITA...,37.28,35004
1,Haría,2021,Hombres,I. CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITA...,0.00,35010
2,San Bartolomé,2021,Hombres,I. CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITA...,21.09,35018
3,Teguise,2021,Hombres,I. CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITA...,17.14,35024
4,Tías,2021,Hombres,I. CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITA...,46.80,35028
...,...,...,...,...,...,...
9513,Tacoronte,2019,Mujeres,XX. CAUSAS EXTERNAS DE MORBILIDAD Y DE MORTALIDAD,32.85,38043
9514,El Tanque,2019,Mujeres,XX. CAUSAS EXTERNAS DE MORBILIDAD Y DE MORTALIDAD,0.00,38044
9515,Tegueste,2019,Mujeres,XX. CAUSAS EXTERNAS DE MORBILIDAD Y DE MORTALIDAD,0.00,38046
9516,La Victoria de Acentejo,2019,Mujeres,XX. CAUSAS EXTERNAS DE MORBILIDAD Y DE MORTALIDAD,0.00,38051


### `df_pob`

In [7]:
df_pob_2019: pd.DataFrame = gpd.read_file('../data/poblacion/Municipios-20190101.json')

# Seleccionamos unicamente las columnas de interes
df_pob_2019 = df_pob_2019.filter([
    'ano',
    'municipio',
    'poblacion',
    'poblacion_hombres',
    'poblacion_mujeres',
    'poblacion_edad_media',
    'geometry'
])

display(df_pob_2019)

,ano,municipio,poblacion,poblacion_hombres,poblacion_mujeres,poblacion_edad_media,geometry
0,2019,35001,5586,2792,2794,45.356427,"MULTIPOLYGON (((-15.74600 28.05110, -15.74580 ..."
1,2019,35002,31619,15984,15635,39.903428,"MULTIPOLYGON (((-15.38690 27.90250, -15.38690 ..."
2,2019,35003,12461,6601,5860,42.225544,"MULTIPOLYGON (((-13.91950 28.25150, -13.91950 ..."
3,2019,35005,1069,589,480,50.978017,"MULTIPOLYGON (((-15.74960 27.98800, -15.74970 ..."
4,2019,35006,38138,19000,19138,42.477686,"MULTIPOLYGON (((-15.55940 28.14720, -15.55940 ..."
...,...,...,...,...,...,...,...
83,2019,38049,4564,2301,2263,46.897458,"MULTIPOLYGON (((-17.34200 28.15370, -17.34070 ..."
84,2019,38052,1667,857,810,46.269646,"MULTIPOLYGON (((-16.66040 28.20140, -16.65990 ..."
85,2019,38053,4843,2453,2390,45.232604,"MULTIPOLYGON (((-17.84060 28.59660, -17.84010 ..."
86,2019,38901,1870,987,883,48.233155,"MULTIPOLYGON (((-18.15700 27.71180, -18.15700 ..."


In [8]:
df_pob_2020: pd.DataFrame = gpd.read_file('../data/poblacion/Municipios-20200101.json')

# Seleccionamos unicamente las columnas de interes
df_pob_2020 = df_pob_2020.filter([
    'ano',
    'municipio',
    'poblacion',
    'poblacion_hombres',
    'poblacion_mujeres',
    'poblacion_edad_media',
    'geometry'
])

display(df_pob_2020)

,ano,municipio,poblacion,poblacion_hombres,poblacion_mujeres,poblacion_edad_media,geometry
0,2020,35001,5599,2800,2799,45.645562,"MULTIPOLYGON (((-15.74600 28.05110, -15.74580 ..."
1,2020,35002,31914,16120,15794,40.297863,"MULTIPOLYGON (((-15.38690 27.90250, -15.38690 ..."
2,2020,35003,12972,6845,6127,42.781992,"MULTIPOLYGON (((-13.91950 28.25150, -13.91950 ..."
3,2020,35005,1049,586,463,51.162536,"MULTIPOLYGON (((-15.74960 27.98800, -15.74970 ..."
4,2020,35004,64645,32443,32202,38.877044,"MULTIPOLYGON (((-13.58360 28.95200, -13.58360 ..."
...,...,...,...,...,...,...,...
83,2020,38051,9158,4563,4595,43.180716,"MULTIPOLYGON (((-16.48760 28.43760, -16.48750 ..."
84,2020,38052,1715,878,837,46.381633,"MULTIPOLYGON (((-16.66040 28.20140, -16.65990 ..."
85,2020,38053,4859,2440,2419,45.480654,"MULTIPOLYGON (((-17.84060 28.59660, -17.84010 ..."
86,2020,38901,1894,993,901,48.553854,"MULTIPOLYGON (((-18.15700 27.71180, -18.15700 ..."


In [9]:
df_pob_2021: pd.DataFrame = gpd.read_file('../data/poblacion/Municipios-20210101.json')

# Seleccionamos unicamente las columnas de interes
df_pob_2021 = df_pob_2021.filter([
    'ano',
    'municipio',
    'poblacion',
    'poblacion_hombres',
    'poblacion_mujeres',
    'poblacion_edad_media',
    'geometry'
])

display(df_pob_2021)

,ano,municipio,poblacion,poblacion_hombres,poblacion_mujeres,poblacion_edad_media,geometry
0,2021,35001,5695,2856,2839,45.887533,"MULTIPOLYGON (((-15.74600 28.05110, -15.74580 ..."
1,2021,35002,32105,16197,15908,40.724451,"MULTIPOLYGON (((-15.38690 27.90250, -15.38690 ..."
2,2021,35003,12783,6716,6067,43.296292,"MULTIPOLYGON (((-13.91950 28.25150, -13.91950 ..."
3,2021,35006,38535,19155,19380,43.182964,"MULTIPOLYGON (((-15.55940 28.14720, -15.55940 ..."
4,2021,35007,762,394,368,45.545932,"MULTIPOLYGON (((-14.05970 28.37750, -14.06100 ..."
...,...,...,...,...,...,...,...
83,2021,38050,2917,1519,1398,52.394755,"MULTIPOLYGON (((-17.24500 28.11170, -17.24510 ..."
84,2021,38051,9161,4568,4593,43.413547,"MULTIPOLYGON (((-16.48760 28.43760, -16.48750 ..."
85,2021,38052,1789,907,882,46.479877,"MULTIPOLYGON (((-16.66040 28.20140, -16.65990 ..."
86,2021,38053,4875,2453,2422,45.805231,"MULTIPOLYGON (((-17.84060 28.59660, -17.84010 ..."


In [10]:
df_pob = pd.concat([
    df_pob_2019,
    df_pob_2020,
    df_pob_2021
])

display(df_pob)

,ano,municipio,poblacion,poblacion_hombres,poblacion_mujeres,poblacion_edad_media,geometry
0,2019,35001,5586,2792,2794,45.356427,"MULTIPOLYGON (((-15.74600 28.05110, -15.74580 ..."
1,2019,35002,31619,15984,15635,39.903428,"MULTIPOLYGON (((-15.38690 27.90250, -15.38690 ..."
2,2019,35003,12461,6601,5860,42.225544,"MULTIPOLYGON (((-13.91950 28.25150, -13.91950 ..."
3,2019,35005,1069,589,480,50.978017,"MULTIPOLYGON (((-15.74960 27.98800, -15.74970 ..."
4,2019,35006,38138,19000,19138,42.477686,"MULTIPOLYGON (((-15.55940 28.14720, -15.55940 ..."
...,...,...,...,...,...,...,...
83,2021,38050,2917,1519,1398,52.394755,"MULTIPOLYGON (((-17.24500 28.11170, -17.24510 ..."
84,2021,38051,9161,4568,4593,43.413547,"MULTIPOLYGON (((-16.48760 28.43760, -16.48750 ..."
85,2021,38052,1789,907,882,46.479877,"MULTIPOLYGON (((-16.66040 28.20140, -16.65990 ..."
86,2021,38053,4875,2453,2422,45.805231,"MULTIPOLYGON (((-17.84060 28.59660, -17.84010 ..."


## Historia

In [ ]:
display(df_defun.query("Municipio == 'Adeje' and Año == 2019"))

display(df_defun.query('Municipio == "Adeje"')['TERRITORIO_CODE'].unique())

display(df_pob.query('municipio == "38001"'))